In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

The source of this type of study is from: https://medium.com/pizzadedados/kmeans-e-metodo-do-cotovelo-94ded9fdf3a9

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
iris = pd.read_csv("../input/iris-flower-dataset/IRIS.csv")

In [ ]:
iris.head()

This is the classic iris dataset with 4 features that can help to classify the specie of flower. I am going to take out the last column of species and check how many clusters (species) I could find with the K-means model. This is a study notebook.

In [ ]:
iris['species'].value_counts()

In [ ]:
iris_df = iris.drop(columns = 'species')
iris_df.head()

In [ ]:
iris_df.shape

In [ ]:
iris_df.info()

In [ ]:
iris_df.describe()

In [ ]:
#Correlation Matrix
iris_df.corr().style.format("{:.2}").background_gradient(cmap=plt.get_cmap("coolwarm"), axis = 1)

The highest correlation is between the petal_length and the petal_width, the second is the petal_length and the sepal_length

In [ ]:
sns.pairplot(iris_df)

These charts confirm the high correlation between the petal_width and the petal_length

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
#Create the model and use the default number of clusters
kmeans = KMeans(random_state = 42)
kmeans.fit(iris_df)

In [ ]:
#Save the centroids value
centroids = kmeans.cluster_centers_

In [ ]:
#Make the prediction
y_kmeans = kmeans.predict(iris_df)

In [ ]:
#Elbow Method
list_clusters = list(range(1,9)) #until the kmeans default clusters number
inertia = []
for number_clusters in list_clusters:
    kmeans = KMeans(n_clusters = number_clusters, random_state = 42)
    kmeans.fit(iris_df)
    inertia.append(kmeans.inertia_)

#Plot the elbow chart
plt.plot(list_clusters, inertia, marker = "+", linestyle = 'solid', mec = 'red' )
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In this chart we can notice that if we choose one cluster (it doesn't make sense, but just for the analysis) the inercia value is  disparate and higher than 2 clusters set, it means that the data is sparse. The goal of clustering is to have the lower inertia for having more compact clusters.
Just to remember --> Inertia: it is the within-clusters sum-of-squares distance.

Which number of clusters is the best for our data? Just looking at the elbow method, It seems that 2 or 3 clusters are good, but it is better to make another aproach to choose this number
Let's try to find the optimal number of clusters

In [ ]:
#Elbow Method STARTING WITH 2 CLUSTERS
list_clusters = list(range(2,9)) #until the kmeans default clusters number
inertia = []
for number_clusters in list_clusters:
    kmeans = KMeans(n_clusters = number_clusters, random_state = 42)
    kmeans.fit(iris_df)
    inertia.append(kmeans.inertia_)

#Plot the elbow chart
plt.plot(list_clusters, inertia, marker = "+", linestyle = 'solid', mec = 'red' )
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

Now it seems that 3 or 4 clusters are the ideal number.

The ideal point where we will find the unity within clusters and the biggest difference between them it is the elbow point further from the straight line created by the points with the minimum and maximum number of clusters that we choose before (2 and 8). The formula of the distance between a line and a point is: https://en.wikipedia.org/wiki/Distance_from_a_point_to_a_line

In [ ]:
# Straight line between 2 clusters and 8 clusters
x0, y0 = 2, inertia[0]
x1, y1 = 8, inertia[-1]
X = [x0, x1]
Y = [y0, y1]
#Plot the elbow chart
plt.plot(list_clusters, inertia, marker = "+", linestyle = 'solid', mec = 'red' )
plt.plot(X, Y)
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.legend(['Elbow method', 'Straight line between clusters'])
plt.show()

In [ ]:
inertia

In [ ]:
list_clusters

In [ ]:
#Make the distance function and decide the best number of clusters
from math import sqrt
def number_of_clusters(list_clusters, inertia):
    distances = []
    dictionary = {}
    x0, y0 = list_clusters[0], inertia[0]
    x1, y1 = list_clusters[-1], inertia[-1]
    i = 0
    for i in range(len(list_clusters)):
        x = list_clusters[i]
        y = inertia[i]
        a = abs((y1-y0)*x - (x1 - x0)*y + (x1*y0) - (y1*x0))
        b = sqrt((y1-y0)**2 + (x1-x0)**2)
        distances.append(a/b)
        dictionary = dict(zip(distances, list_clusters))
        i +=1     
    return print('Optimal number of clusters is:', dictionary.get(max(dictionary.keys())), "\nDistances between the point and the straight line: ", distances)

In [ ]:
number_of_clusters(list_clusters, inertia)

In [ ]:
#Create the model again but with n_clusters = 4 and adding a new column to the DataFrame
kmeans = KMeans(n_clusters = 4, random_state = 42)
kmeans.fit(iris_df)
iris_df['clusters'] = kmeans.predict(iris_df)

In [ ]:
iris_df.head()

In [ ]:
iris_df['clusters'].value_counts()

In [ ]:
sns.pairplot(iris_df, hue = 'clusters')

Cluster 0 --> It is really a separated group with the lower values of petal_width, petal_length.
Cluster 1, 2, 3 --> They are mixed and in scale, the values of cluster 1 are lower for petal_width and petal_length than the others and so on.
Let's compare these result with the real classification of the real iris dataset

In [ ]:
#cluster chart of the  petal_length and the petal_width
plt.subplots(figsize = (12,7))
plt.subplot(1,2,1)
plt.title('K-means result')
plt.scatter(iris_df['petal_length'], iris_df['petal_width'], c = iris_df['clusters'], cmap = 'winter')
plt.xlabel('petal_length')
plt.ylabel('petal_width')

plt.subplot(1,2,2)
plt.title('Real classification')
plt.scatter(iris['petal_length'], iris['petal_width'], c = iris['species'].replace(iris['species'].value_counts().index, [0,1,2]), cmap = 'winter')
plt.xlabel('petal_length')
plt.ylabel('petal_width')

As you can see, our model found more clusters than the real dataset. It happened because the point distances of 3 clusters and 4 clusters from the straight line were very similar (2.599709771898713, 2.66059664173226), and the inertia value of 4 clusters is less than 3 clusters.
If we pay attention to the number of samples inside each cluster, the cluster 0 has 50, the cluster 1 has 40, the cluster 2 has 32, the cluster 3 has 28 - from that info when we divide the dataset into a cluster we expect to have similar numbers of samples, and the clusters 2 and 3 have less samples than the others. 
Maybe, if we choose more clusters to do the elbow method, the straight line and the point position could change and the result clould be closer to the real. But the majority of the unsupervised models will not have the "answer" as we have here, and the next part of the method could be study more about these clusters 2 and 3 and see if they really make sense to belong to theses clusters or its better to join them somehow.